In [0]:
%run /Workspace/Users/dataeng.victor_outlook.com#ext#@dataengvictoroutlook.onmicrosoft.com/01.config/00.global_config


In [0]:
%run /Workspace/Repos/dataeng.victor@outlook.com/databricks-weather-realtime/03.silver/01_silver_transform

In [0]:
## Resgatando o schema do JSON da bronze
my_schema = (
    spark.table(f"{catalog}.{bronze['schema']}.{bronze['table']}")
         .select("json_str")
         .limit(1)
         .collect()[0]["json_str"]
)
my_schema


In [0]:
from pyspark.sql.functions import schema_of_json

auto_schema = schema_of_json(my_schema)
print(auto_schema)


In [0]:
#Ler, parsear JSON e estruturar
from pyspark.sql.functions import *

bronze_table = f"{catalog}.{bronze['schema']}.{bronze['table']}"

df_bronze = spark.read.format("delta").table(bronze_table)

df_silver = (
    df_bronze
        .withColumn("json_parsed", from_json(col("json_str"), auto_schema))
        .select(
            col("json_parsed.data.time").alias("timestamp"),
            col("json_parsed.data.values.temperature").alias("temperature"),
            col("json_parsed.data.values.temperatureApparent").alias("temperature_apparent"),
            col("json_parsed.data.values.humidity").alias("humidity"),
            col("json_parsed.data.values.windSpeed").alias("wind_speed"),
            col("json_parsed.data.values.windGust").alias("wind_gust"),
            col("json_parsed.data.values.windDirection").alias("wind_direction"),
            col("json_parsed.data.values.cloudCover").alias("cloud_cover"),
            col("json_parsed.data.values.visibility").alias("visibility"),
            col("json_parsed.data.values.dewPoint").alias("dew_point"),
            col("json_parsed.data.values.pressureSeaLevel").alias("pressure_sea_level"),
            col("json_parsed.data.values.altimeterSetting").alias("altimeter"),
            col("json_parsed.location.lat").alias("lat"),
            col("json_parsed.location.lon").alias("lon"),
            col("json_parsed.location.name").alias("location_name")
        )
)


In [0]:
df_silver.write.mode("overwrite").format("delta").saveAsTable(
    f"{catalog}.{silver['schema']}.{silver['table']}"
)


In [0]:
%sql
SELECT * FROM weather_realtime.silver.weather_clean;
